<a href="https://colab.research.google.com/github/lyeslehara1996/PFE/blob/main/Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install np_utils

     |████████████████████████████████| 61kB 2.2MB/s 
  Created wheel for np-utils: filename=np_utils-0.5.12.1-cp37-none-any.whl size=57133 sha256=6ead89dfecc3b51b068861871cd918f68dd1d240fd259caf7ef44440f7c552a1
  Stored in directory: /root/.cache/pip/wheels/92/4b/81/206efd0d01330a96f3aebe5021d2d5f0b264b7ade827c306ef
Successfully built np-utils


In [16]:
#declaration des module 

import pandas as pd 
import re 
import nltk

import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dropout

from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import TweetTokenizer
import string as st
SAVEd = False

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,classification_report
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from keras.utils.np_utils import to_categorical

In [18]:
df=pd.read_excel('/content/SemEval2017.xlsx')

netoyage de dataset



In [19]:
df.drop("Unnamed: 3", axis=1, inplace=True)
df.drop("Unnamed: 4", axis=1, inplace=True)
df.drop("Unnamed: 5", axis=1, inplace=True)
df.drop("Unnamed: 6", axis=1, inplace=True)
df.drop("Unnamed: 7", axis=1, inplace=True)
df.drop("Unnamed: 8", axis=1, inplace=True)
df.drop("Unnamed: 9", axis=1, inplace=True)
df.drop("Unnamed: 10", axis=1, inplace=True)
df.drop("Unnamed: 11", axis=1, inplace=True)
df.drop("Unnamed: 12", axis=1, inplace=True)
df.drop("Unnamed: 13", axis=1, inplace=True)
df.drop("Unnamed: 14", axis=1, inplace=True)
df.drop("Unnamed: 15", axis=1, inplace=True)
df.head()

,id,polariy,Comments
0,619950566786113024,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T..."
1,619969366986235008,neutral,Order Go Set a Watchman in store or through ou...
2,619971047195044992,negative,If these runway renovations at the airport pre...
3,619974445185302016,neutral,If you could ask an onstage interview question...
4,619987808317406976,positive,A portion of book sales from our Harper Lee/Go...


**Prétraitement de dataSet**

transformer les mots en miniscule


In [20]:
df_clean = df
df_clean.Comments=df_clean.Comments.str.lower()
df_clean



,id,polariy,Comments
0,619950566786113024,neutral,"picturehouse's, pink floyd's, 'roger waters: t..."
1,619969366986235008,neutral,order go set a watchman in store or through ou...
2,619971047195044992,negative,if these runway renovations at the airport pre...
3,619974445185302016,neutral,if you could ask an onstage interview question...
4,619987808317406976,positive,a portion of book sales from our harper lee/go...
...,...,...,...
20628,681877834982232064,neutral,@shaquillehoneal from what i think you're aski...
20629,681879579129200000,positive,"iran ranks 1st in liver surgeries, allah bless..."
20630,681883903259357056,neutral,hours before he arrived in saudi arabia on tue...
20631,681904976860327040,negative,@vanityfair alex kim kardashian worth how to ...


Suppression de tout les symbole  

In [21]:
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r'https?:\/\/\S+', ' ', str(x)))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", ' ', str(x)))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r'{link}', ' ', str(x)))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r'&[a-z]+;', ' ', str(x)))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r"[^a-z]", ' ', str(x)))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r'@mention', ' ', str(x)))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: " ".join(x.lower() for x in str(x).split()  if len(x)>3 ))
df_clean['Comments'].head()

0    picturehouse pink floyd roger waters walll ope...
1    order watchman store through website before tu...
2    these runway renovations airport prevent from ...
3    could onstage interview question miss tomorrow...
4    portion book sales from harper watchman releas...
Name: Comments, dtype: object

In [22]:
df_clean['polariy'] = df_clean['polariy'].apply(lambda x: re.sub(r'https?:\/\/\S+><', ' ', str(x)))


In [23]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

Suppression de stop words


In [24]:
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
df_clean['Comments']=df_clean['Comments'].apply(remove_stopwords)

{'there', 'd', 'above', 's', 'needn', "hadn't", 'have', 'when', 'all', 'ma', "should've", 'himself', 'being', "wouldn't", "won't", 'with', 'just', 'his', 'but', 'm', 'an', "aren't", 'down', 'under', 'we', 'more', 'this', 'by', "didn't", 'itself', 'you', "you'd", 'few', 'they', 'between', 'ain', 'too', 'ours', 'any', 'where', 'the', 'how', 'against', "needn't", 'after', 'her', 'each', 'mustn', 'for', "shouldn't", 'myself', "couldn't", 'o', "mustn't", 'most', 'why', "you'll", 'who', 'of', 'not', 'him', 'isn', 'our', 'weren', 't', 'yourself', 'off', 'some', 're', 'he', 'doing', 'can', 'theirs', 'at', "that'll", 'then', 'it', 'are', "don't", 'other', 've', 'aren', 'during', "shan't", 'were', "you're", 'a', 'while', 'once', 'wasn', 'and', 'own', 'y', 'whom', 'because', "hasn't", "haven't", 'in', "mightn't", 'that', 'up', 'she', 'nor', 'my', 'your', 'both', 'wouldn', 'over', 'mightn', 'was', 'hers', 'me', 'is', 'been', 'into', 'what', 'yours', 'herself', 'so', 'them', "isn't", 'won', 'does',

Décomposition de dataset

In [79]:
reviews =  df_clean[['Comments']]
labels =  df_clean[['polariy']]

suppression de ponctuation

In [78]:
revue_sans_ponctuation=[]
for sentence in reviews['Comments']:

    revue_sans_ponctuation.append(' '.join(Word.strip(st.punctuation) for Word in sentence.split()))


In [64]:
reviews_cleaned = np.asarray(revue_sans_ponctuation)
reviews_cleaned

array(['picturehouse pink floyd roger waters walll opening sept making waves watch trailer rolling stone look',
       'order watchman store website tuesday half price gsaw gsawatchmanbook',
       'runway renovations airport prevent seeing taylor swift monday blood meaning',
       ...,
       'hours arrived saudi arabia tuesday turkish president recep tayyip erdogan accused syria president mercilessly',
       'vanityfair alex kardashian worth love kardashian conure',
       'guess even pandora knows justin bieber grown condom played'],
      dtype='<U121')

In [81]:
review_array = np.asarray(reviews)
label_array = np.asarray(labels)

reviews_labels = np.stack((review_array, label_array), axis = 1)

In [82]:
list_index=[]

for i, text in enumerate(reviews_labels[:,0:1]):
    if(text == "\n"):
        list_index.append(i)

reviews_labels = np.delete(reviews_labels, list_index, axis=0)
reviews_labels

array([[['picturehouse pink floyd roger waters walll opening sept making waves watch trailer rolling stone look'],
        ['neutral']],

       [['order watchman store website tuesday half price gsaw gsawatchmanbook'],
        ['neutral']],

       [['runway renovations airport prevent seeing taylor swift monday blood meaning'],
        ['negative']],

       ...,

       [['hours arrived saudi arabia tuesday turkish president recep tayyip erdogan accused syria president mercilessly'],
        ['neutral']],

       [['vanityfair alex kardashian worth love kardashian conure'],
        ['negative']],

       [['guess even pandora knows justin bieber grown condom played'],
        ['neutral']]], dtype=object)

Encoder les labels multi class

In [104]:


le=LabelEncoder()
encoded_labels= le.fit_transform( label_array)
encoded_labels = to_categorical(encoded_labels)
encoded_labels

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

Déviser les donnés de test et d'entraînement

In [105]:
review_train, review_test, label_train, label_test = train_test_split(reviews_cleaned,encoded_labels ,test_size=0.2, random_state=42)

print(review_train.shape, label_train.shape)
print(review_test.shape, label_test.shape)


(16506,) (16506, 4)
(4127,) (4127, 4)


Encoder les reviews

In [106]:
tokenizer = Tokenizer(num_words=4000)
tokenizer.fit_on_texts(review_train)

review_train = tokenizer.texts_to_sequences(review_train)
review_test = tokenizer.texts_to_sequences(review_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

review_train = pad_sequences(review_train, padding='post', maxlen=maxlen)
review_test = pad_sequences(review_test, padding='post', maxlen=maxlen)

In [107]:
review_train.shape
review_train

array([[100,  19,  20, ...,   0,   0,   0],
       [ 13,  77, 526, ...,   0,   0,   0],
       [231, 124, 128, ...,   0,   0,   0],
       ...,
       [ 94,  83, 529, ...,   0,   0,   0],
       [ 55,  36, 131, ...,   0,   0,   0],
       [568,   5,  98, ...,   0,   0,   0]], dtype=int32)

Model 


In [108]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(maxlen,3)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [109]:
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 100, 50)           10800     
_________________________________________________________________
dropout_15 (Dropout)         (None, 100, 50)           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 100, 50)           20200     
_________________________________________________________________
dropout_16 (Dropout)         (None, 100, 50)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 100, 50)           20200     
_________________________________________________________________
dropout_17 (Dropout)         (None, 100, 50)           0         
Total params: 51,200
Trainable params: 51,200
Non-trainable params: 0
__________________________________________________

In [110]:
  history=model.fit(review_train,label_train, epochs=100,batch_size=64,validation_data=(review_test,label_test))  

Epoch 1/100


ValueError: ignored